In [16]:
!pip install azure-cognitiveservices-vision-customvision


     |████████████████████████████████| 112kB 5.1MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [17]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region

ENDPOINT = "https://westeurope.api.cognitive.microsoft.com/"

# Replace with a valid key
training_key = "ee96ce4c988e46618994b8fba546326f"
prediction_key = "ee96ce4c988e46618994b8fba546326f"
prediction_resource_id = "/subscriptions/d0c96ed6-4a68-4c71-96d1-08b497bca696/resourceGroups/customvision/providers/Microsoft.CognitiveServices/accounts/customvision"

publish_iteration_name = "detectModel"

trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
project = trainer.create_project("My Detection Project", domain_id=obj_detection_domain.id)

Creating project...


In [18]:
raim_tag = trainer.create_tag(project.id, "raim")

In [19]:
import os
import pandas as pd

dfall=pd.read_csv("./all.list",header=None, sep=' ')
dfall.columns=['f']

dfall

,f
0,CDY_2015
1,CDY_2016
2,CDY_2017
3,CDY_2018
4,CDY_20180427_152724818_BURST000_COVER_TOP
5,CDY_20180427_152823935_BURST000_COVER_TOP
6,CDY_20180427_152937457_BURST000_COVER_TOP
7,CDY_20180427_153021423_BURST001
8,CDY_20180427_153126820_BURST000_COVER_TOP
9,CDY_20180427_153126820_BURST001


In [24]:
import os
import pandas as pd
import numpy as np
from io import StringIO
import requests

base_image_url = 'https://raw.githubusercontent.com/nowheremanmail/wgisd/master/data/'

# Go through the data table above and create the images
print ("Adding images...")
tagged_images_with_regions = []

for file_names in dfall.values:
    file_name = file_names[0]
    print (file_name)
    regions = []

    s=base_image_url + file_name + '.txt'
    df=pd.read_csv(s,header=None, sep=' ')
    df.columns=['c', 'x','y', 'w', 'h']

    #print (df.values)

    regions = []
    for i in df.values:
         regions += [ Region(tag_id=raim_tag.id, left=i[1]-i[3]/2,top=i[2]-i[4]/2,width=i[3],height=i[4]) ]

    #print(regions)
    ss = base_image_url + file_name + '.jpg'
    image_contents = requests.get(ss).content
    tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents, regions=regions))

    if len(tagged_images_with_regions) >= 50:
        upload_result = trainer.create_images_from_files(project.id, images=tagged_images_with_regions)
        if not upload_result.is_batch_successful:
            print("Image batch upload failed.")
            for image in upload_result.images:
                print("Image status: ", image.status)
        tagged_images_with_regions = []
        
                
if len(tagged_images_with_regions) > 0:
    upload_result = trainer.create_images_from_files(project.id, images=tagged_images_with_regions)
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)


Adding images...
CDY_2015
CDY_2016
CDY_2017
CDY_2018
CDY_20180427_152724818_BURST000_COVER_TOP
CDY_20180427_152823935_BURST000_COVER_TOP
CDY_20180427_152937457_BURST000_COVER_TOP
CDY_20180427_153021423_BURST001
CDY_20180427_153126820_BURST000_COVER_TOP
CDY_20180427_153126820_BURST001
CDY_20180427_153144437_BURST000_COVER_TOP
CDY_20180427_153152483_BURST001
CDY_20180427_153201477_BURST000_COVER_TOP
CDY_20180427_153310519_BURST000_COVER_TOP
CDY_20180427_153409150_BURST000_COVER_TOP
CDY_20180427_153421615_BURST001
CDY_20180427_153433120_BURST001
CDY_20180427_153557690_BURST000_COVER_TOP
CDY_20180427_153615626_BURST000_COVER_TOP
CDY_2019
CDY_2020
CDY_2021
CDY_2022
CDY_2023
CDY_2024
CDY_2025
CDY_2026
CDY_2027
CDY_2028
CDY_2029
CDY_2030
CDY_2031
CDY_2032
CDY_2033
CDY_2034
CDY_2035
CDY_2036
CDY_2037
CDY_2038
CDY_2039
CDY_2040
CDY_2041
CDY_2042
CDY_2043
CDY_2044
CDY_2045
CDY_2046
CDY_2047
CDY_2048
CDY_2049
CDY_2050
CDY_2051
CDY_2052
CDY_2053
CDY_2054
CDY_2055
CDY_2056
CDY_2057
CDY_2058
CDY_205